In [11]:
%reload_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

from dmsensei import DataModule, create_model, Dataset
from dmsensei.config import device
from dmsensei.core.callbacks import WandbFitLogger, KaggleLogger
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch import Trainer
from dmsensei.config import device
import sys
import os
from lightning.pytorch.loggers import WandbLogger
import torch, wandb
import numpy as np


dm = DataModule(
    name=["ribo500-blast"],#, 'sarah_supermodel'],
    data_type=["dms", 'shape', 'structure'],
    force_download=False,
    batch_size=64,
    num_workers=0,
    train_split=128,
    valid_split=0,
    predict_split=0,
    overfit_mode=True,
    shuffle_valid=False,
    use_error=False,
    structure_padding_value=0,
)

dm.setup('fit')

Loading dataset from disk
Done!                            


In [8]:
from dmsensei import create_model

params = {
    'embed_dim': 192,
    'num_heads': 6,
    'hidden_dim': 768,
    'num_encoders': 12,
    'max_len': 500,
    'use_se': True,
    
}
params['dim_per_head'] = params['embed_dim'] // params['num_heads']

model = create_model(
    'ribonanza',
    params,
    lr = 1e-3,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.MSELoss()
for batch in dm.train_dataloader():
    pred = model.forward(batch)['dms']
    pairs = [(p.to(device), t) for p, t in zip(pred, batch.get('dms')) if t is not None]
    print(pairs[0][0].device, pairs[0][1].device)
    optimizer.zero_grad()
    loss = torch.tensor(0.)
    for p, t in pairs:
        if t is None:
            continue
        p = p[1:-1]
        mask = t != -1000.
        if not torch.sum(mask):
            continue
        p, t = p[mask], t[mask]
        if p.device != t.device:
            print('mismatch')
        loss += criterion(p, t)
    loss.backward()
    

    break

cuda:0 cuda:0


IndexError: The shape of the mask [177] at index 0 does not match the shape of the indexed tensor [175] at index 0

In [8]:
from torch.utils.data import Sampler
import numpy as np
from random import shuffle

class BySequenceLengthSampler(Sampler):

    def __init__(self, dataset,  
                bucket_boundaries, batch_size=64,):
        ind_n_len = []
        for i, p in enumerate(dataset.length):
            ind_n_len.append( (i, p) )
        self.ind_n_len = ind_n_len
        self.bucket_boundaries = bucket_boundaries
        self.batch_size = batch_size
        self.data_source = dataset
        
    def __iter__(self):
        data_buckets = dict()
        # where p is the id number and seq_len is the length of this id number. 
        for p, seq_len in self.ind_n_len:
            pid = self.element_to_bucket_id(p,seq_len)
            if pid in data_buckets.keys():
                data_buckets[pid].append(p)
            else:
                data_buckets[pid] = [p]

        for k in data_buckets.keys():

            data_buckets[k] = np.asarray(data_buckets[k])

        iter_list = []
        for k in data_buckets.keys():
            np.random.shuffle(data_buckets[k])
            iter_list += (np.array_split(data_buckets[k]
                           , int(data_buckets[k].shape[0]/self.batch_size)))
        shuffle(iter_list) # shuffle all the batches so they arent ordered by bucket
        # size
        for i in iter_list: 
            yield i.tolist() # as it was stored in an array
    
    def __len__(self):
        return len(self.data_source)
    
    def element_to_bucket_id(self, x, seq_length):
        boundaries = list(self.bucket_boundaries)
        buckets_min = [np.iinfo(np.int32).min] + boundaries
        buckets_max = boundaries + [np.iinfo(np.int32).max]
        conditions_c = np.logical_and(
          np.less_equal(buckets_min, seq_length),
          np.less(seq_length, buckets_max))
        bucket_id = np.min(np.where(conditions_c))
        return bucket_id
    
# To call it and use it in a DataLoader (Replace the batch_size and bucket_boundaries values):



In [9]:
from dmsensei import DataModule, create_model, Dataset
from torch.utils.data import DataLoader, random_split, Subset

ds = Dataset.from_local_or_download(
    name = 'yack_valid',
)

bucket_boundaries = [50,100,125,150,175,200,250,300]
batch_sizes=32

dataloader = DataLoader(ds, batch_size=1, 
                        batch_sampler=sampler, 
                        num_workers=0, 
                        
                        drop_last=False, pin_memory=False)


Loading dataset from disk
Done!                            


In [10]:
for batch in dataloader:
    print(batch['dms'].shape)
    break

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [7]:
ds.length

[177,
 94,
 75,
 177,
 73,
 89,
 17,
 120,
 177,
 127,
 87,
 177,
 177,
 142,
 177,
 140,
 75,
 84,
 177,
 177,
 177,
 73,
 102,
 177,
 184,
 79,
 177,
 86,
 177,
 358,
 89,
 140,
 177,
 177,
 219,
 61,
 69,
 117,
 177,
 123,
 177,
 104,
 143,
 177,
 87,
 177,
 102,
 177,
 177,
 177,
 58,
 177,
 115,
 120,
 155,
 177,
 476,
 177,
 177,
 104,
 72,
 159,
 41,
 177,
 148,
 177,
 177,
 78,
 177,
 77,
 399,
 177,
 179,
 158,
 177,
 77,
 177,
 177,
 91,
 120,
 84,
 177,
 420,
 155,
 122,
 82,
 115,
 177,
 177,
 90,
 177,
 76,
 81,
 122,
 177,
 177,
 177,
 136,
 177,
 177,
 82,
 71,
 117,
 177,
 178,
 40,
 75,
 177,
 85,
 177,
 138,
 72,
 74,
 119,
 117,
 218,
 177,
 83,
 132,
 180,
 177,
 129,
 177,
 78,
 77,
 115,
 177,
 177,
 114,
 30,
 177,
 177,
 177,
 177,
 75,
 177,
 124,
 164,
 117,
 118,
 77,
 75,
 71,
 70,
 177,
 177,
 436,
 177,
 170,
 80,
 177,
 134,
 177,
 119,
 177,
 83,
 76,
 100,
 177,
 18,
 177,
 118,
 325,
 58,
 73,
 177,
 52,
 177,
 177,
 78,
 110,
 184,
 122,
 62,
 77,
 75